<a href="https://colab.research.google.com/github/coffeemountain/kaggle_otto/blob/main/notebooks/create_candidates_from_chris_rerank_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 概要
- step1のcandidate作成とその評価を行うnotebook
- session × aid のcandidateのdataframを作成
- 評価は、recallで作成したTopK個の候補に、正解の20個のうち何個が含まれているか。で計算される

## 使い方
- generate_func_sampleを読んで、候補作成メインロジックを理解
- CandidateConfigに自分の作成したいcandidateの条件を入力

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#@title utils.ipynbから必要なcomponentをダウンロード
%cd /content/drive/MyDrive/kaggle/2022/OTTO/share
! pip install import-ipynb
import import_ipynb
from utils import calc_recall, CovisMatrixConfig, CovisMatrixGenerator, CovisMatrixWeightFuncMixin, CandidateConfig, pqt_to_dict, to_submission_from_exploded

/content/drive/.shortcut-targets-by-id/1H8a2MOay2LUWBmYLaUl2SUzTIGsFdpHT/share
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/pip-install.py", line 17, in <module>
    pynvml.nvmlInit()
  File "/usr/local/lib/python3.8/dist-packages/pynvml/nvml.py", line 1450, in nvmlInit
    nvmlInitWithFlags(0)
  File "/usr/local/lib/python3.8/dist-packages/pynvml/nvml.py", line 1440, in nvmlInitWithFlags
    _nvmlCheckReturn(ret)
  File "/usr/local/lib/python3.8/dist-packages/pynvml/nvml.py", line 765, in _nvmlCheckReturn
    raise NVMLError(ret)
pynvml.nvml.NVMLError_DriverNotLoaded: Driver Not Loaded

During handling of the above exception, another e

## candidatesを作成する
- 入力: train_full, train, test
- 出力: train_candidates, test_candidates
  - columns: session, aid


### 1. データの準備
train_full, train, testをロード

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import itertools
from collections import Counter
from tqdm.notebook import tqdm

tqdm.pandas()

In [ ]:
def chris_suggest_clicks(df, topk, covis_matrix_generator, covis_matrix_config, covis_matrix_output_dir, test_df):
  """
  [desc]
  candidateを作成したいデータを渡すと、topk個のcandidateを返却

  [入力]
    - df: candidateを作成した元のデータ
      - columns
        - session
        - aid
        - ts
        - type
    - topk: 各sessionに対して、生成したいcandidateの数
    - target: ターゲットとするevent type

  [出力]
    - candidates: 各sessionに対する候補たち。各sessionに対して、topk個のrowが並ぶ
      - columns
        - session
        - aid
  """
  covis_matrix = covis_matrix_generator.load_or_generate(covis_matrix_config, covis_matrix_output_dir)
  clicks_covis_matrix = pqt_to_dict(covis_matrix)
  print(len(clicks_covis_matrix))
  print('load covis_matrix')

  # FIXME: chrisコードのバグ。typeがint型にmapされているので、該当なし
  top_clicks = test_df.loc[test_df['type'] == 0,'aid'].value_counts().index.values[:topk]

  # ======== chrisのsuggest clicksコード (出力数topkだけ変えられるようにした)================
  # type_weight_multipliers = {'clicks': 1, 'carts': 6, 'orders': 3}
  type_weight_multipliers = {0: 1, 1: 6, 2: 3}
  def suggest_clicks(df):
      # USER HISTORY AIDS AND TYPES
      aids=df.aid.tolist()
      types = df.type.tolist()
      unique_aids = list(dict.fromkeys(aids[::-1] ))
      # RERANK CANDIDATES USING WEIGHTS
      if len(unique_aids)>=20:
          weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
          aids_temp = Counter() 
          # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
          for aid,w,t in zip(aids,weights,types): 
              aids_temp[aid] += w * type_weight_multipliers[t]
          sorted_aids = [k for k,v in aids_temp.most_common(topk)]
          return sorted_aids
      # USE "CLICKS" CO-VISITATION MATRIX
      aids2 = list(itertools.chain(*[clicks_covis_matrix[aid] for aid in unique_aids if aid in clicks_covis_matrix]))
      # RERANK CANDIDATES
      top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(topk) if aid2 not in unique_aids]    
      result = unique_aids + top_aids2[:topk - len(unique_aids)]
      # USE TOP20 TEST CLICKS
      return result + list(top_clicks)[:topk-len(result)]

  # =======================================================================

  df = df.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(lambda x: suggest_clicks(x))
  df.name = 'aid'
  df = df.to_frame().reset_index().explode('aid')
  assert len(df) <= df['session'].nunique() * topk  
  return df


In [ ]:
def chris_suggest_carts_and_orders(df, topk, covis_matrix_generator, buys_config, buys_output_dir, buy2buy_config, buy2buy_output_dir, test_df):

  covis_matrix = covis_matrix_generator.load_or_generate(buys_config, buys_output_dir)
  top_20_buys = pqt_to_dict(covis_matrix)

  covis_matrix = covis_matrix_generator.load_or_generate(buy2buy_config, buy2buy_output_dir)
  top_20_buy2buy = pqt_to_dict(covis_matrix)

  # FIXME: chrisコードのバグ。typeがint型にmapされているので、該当なし
  top_orders = test_df.loc[test_df['type'] == 2,'aid'].value_counts().index.values[:topk]

  type_weight_multipliers = {0: 1, 1: 6, 2: 3}
  def suggest_buys(df):
      # USER HISTORY AIDS AND TYPES
      aids=df.aid.tolist()
      types = df.type.tolist()
      # UNIQUE AIDS AND UNIQUE BUYS
      unique_aids = list(dict.fromkeys(aids[::-1] ))
      df = df.loc[(df['type']==1)|(df['type']==2)]
      unique_buys = list(dict.fromkeys( df.aid.tolist()[::-1] ))
      # RERANK CANDIDATES USING WEIGHTS
      if len(unique_aids)>=20:
          weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
          aids_temp = Counter() 
          # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
          for aid,w,t in zip(aids,weights,types): 
              aids_temp[aid] += w * type_weight_multipliers[t]
          # RERANK CANDIDATES USING "BUY2BUY" CO-VISITATION MATRIX
          aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
          for aid in aids3: aids_temp[aid] += 0.1
          sorted_aids = [k for k,v in aids_temp.most_common(topk)]
          return sorted_aids

      # USE "CART ORDER" CO-VISITATION MATRIX
      aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
      # USE "BUY2BUY" CO-VISITATION MATRIX
      aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
      # RERANK CANDIDATES
      top_aids2 = [aid2 for aid2, cnt in Counter(aids2+aids3).most_common(topk) if aid2 not in unique_aids] 
      result = unique_aids + top_aids2[:topk - len(unique_aids)]
      # USE TOP20 TEST ORDERS
      return result + list(top_orders)[:topk-len(result)]

  df = df.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(
    lambda x: suggest_buys(x)
  )
  df.name = 'aid'
  df = df.to_frame().reset_index().explode('aid')
  assert len(df) <= df['session'].nunique() * topk  
  return df

In [ ]:
DATASET_NAME = 'chris-baseline'
TARGET_DIR = f'/content/drive/MyDrive/kaggle/2022/OTTO/share/{DATASET_NAME}/input'

configs = [
    # lb test clicks用candidate作成
    CandidateConfig(
      input_files=glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/test_parquet/*'),
      output_file=TARGET_DIR + '/candidates/v3/clicks_lb_test_candidates.parquet',
      labels_file = None,
      generate_func=chris_suggest_clicks,
      additional_input_dict={
          'covis_matrix_generator': CovisMatrixGenerator(weight_func_mixin=CovisMatrixWeightFuncMixin(), use_gpu=True),
          'covis_matrix_config': CovisMatrixConfig(
              target_types=[0, 1, 2],
              weight_func='time_weight_v1',
              min_event_threshold=30,
              max_sec_threshold=24 * 60 * 60,
              save_topk=20,
              # Covisを作成する際のデータ
              source_data=glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/*_parquet/*'),
          ),
          'covis_matrix_output_dir': TARGET_DIR + '/covis_matrix/clicks_v1',
          'test_df': pd.concat([pd.read_parquet(f) for f in glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/test_parquet/*')]),
      },
      target='clicks',
      topk=20,
      ),
    # lb test carts用candidate作成
    CandidateConfig(
      input_files=glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/test_parquet/*'),
      output_file=TARGET_DIR + '/candidates/v2/carts_orders_lb_test_candidates.parquet',
      labels_file = None,
      generate_func=chris_suggest_carts_and_orders,
      additional_input_dict={
          'covis_matrix_generator': CovisMatrixGenerator(weight_func_mixin=CovisMatrixWeightFuncMixin(), use_gpu=True),
          'buys_config': CovisMatrixConfig(
              target_types=[0, 1, 2], 
              weight_func='type_weight_1_6_3',
              min_event_threshold=30,
              max_sec_threshold=24 * 60 * 60,
              save_topk=15,
              # Covisを作成する際のデータ
              source_data=glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/*_parquet/*'),
          ),
          'buys_output_dir': TARGET_DIR + '/covis_matrix/buys_v1',
          'buy2buy_config': CovisMatrixConfig(
              target_types=[1, 2],
              weight_func='type_weight_1_1_1',
              min_event_threshold=30,
              max_sec_threshold=14 * 24 * 60 * 60,
              save_topk=15,
              # Covisを作成する際のデータ
              source_data=glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/*_parquet/*'),
          ),
          'buy2buy_output_dir': TARGET_DIR + '/covis_matrix/buy2buy_v1',
          'test_df': pd.concat([pd.read_parquet(f) for f in glob.glob(TARGET_DIR + '/otto-chunk-data-inparquet-format/test_parquet/*')]),
      },
      target=None,
      topk=20,
      ),
]

In [ ]:
for config in configs:
  # 元データのロード
  df = pd.concat([pd.read_parquet(file_) for file_ in tqdm(config.input_files)])

  # candidateを作る
  candidates = config.generate_func(df, config.topk, **config.additional_input_dict)

  # 保存
  output_dir = '/'.join(config.output_file.split('/')[:-1])
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  candidates.to_parquet(config.output_file)

  # labelsファイルが存在している場合
  if config.labels_file:
      # 評価
    pred_labels = candidates.groupby('session')['aid'].apply(list).rename('labels').to_frame().reset_index()
    true_labels = pd.read_parquet(config.labels_file)
    recall = calc_recall(true_labels, pred_labels, config.target, config.topk)
    print(f'{config.target} recall@{config.topk} {recall}')

  # TODO: スプレッドシートへ自動保存

  0%|          | 0/17 [00:00<?, ?it/s]

loading covis matrix


  0%|          | 0/5 [00:00<?, ?it/s]

1837166
load covis_matrix


  0%|          | 0/1671803 [00:00<?, ?it/s]

...

loading covis matrix


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

,session,aid
0,12899779,59625
0,12899779,397451
0,12899779,469285
0,12899779,1493965
0,12899779,438191
...,...,...
1671802,14571581,984794
1671802,14571581,986164
1671802,14571581,1460571
1671802,14571581,329725


,session,aid
0,12899779,59625
0,12899779,397451
0,12899779,469285
0,12899779,1493965
0,12899779,438191
...,...,...
1671802,14571581,984794
1671802,14571581,986164
1671802,14571581,1460571
1671802,14571581,329725


In [ ]:
#@title: このまま提出してみる
pred_df_clicks = pd.read_parquet(TARGET_DIR + '/candidates/v2/clicks_lb_test_candidates.parquet')
pred_df_cartsorders = pd.read_parquet(TARGET_DIR + '/candidates/v2/carts_orders_lb_test_candidates.parquet')

submission_clicks = to_submission_from_exploded(pred_df_clicks, 'clicks')
submission_carts = to_submission_from_exploded(pred_df_cartsorders, 'carts')
submission_orders = to_submission_from_exploded(pred_df_cartsorders, 'orders')

pd.concat([submission_clicks, submission_carts, submission_orders]).to_csv(TARGET_DIR + '/../submissions/chris_submission_v2.csv', index=False)

  0%|          | 0/1671803 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

  0%|          | 0/1671803 [00:00<?, ?it/s]

In [ ]:
submion_df = pd.concat([submission_clicks, submission_carts, submission_orders])

In [ ]:
submion_df

In [ ]:
pred_df_cartsorders.groupby('session').size().unique()

array([20])

In [ ]:
pred_df_clicks.groupby('session').size().unique()

array([20])